In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#https://s3.amazonaws.com/hackerday.datascience/112/test.csv
#https://s3.amazonaws.com/hackerday.datascience/112/train.csv
train_df = pd.read_csv("https://s3.amazonaws.com/hackerday.datascience/112/train.csv")
test_df = pd.read_csv("https://s3.amazonaws.com/hackerday.datascience/112/test.csv")

In [3]:
feat_labels = train_df.head(0)
unique_activities = train_df.Activity.unique()
replacer = {}
for i, activity in enumerate(unique_activities):
    replacer[activity] = i
train_df.Activity = train_df.Activity.replace(replacer)
test_df.Activity = test_df.Activity.replace(replacer)

In [4]:
train_df = train_df.drop("subject", axis=1)
test_df = test_df.drop("subject", axis=1)

def get_all_data():
    train_values = train_df.values
    test_values = test_df.values
    np.random.shuffle(train_values)
    np.random.shuffle(test_values)
    X_train = train_values[:, :-1]
    X_test = test_values[:, :-1]
    y_train = train_values[:, -1]
    y_test = test_values[:, -1]
    return X_train, X_test, y_train, y_test

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = get_all_data()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)

model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)
model.score(X_test, y_test)

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


(999, 561)


0.908908908908909

In [5]:
train_df = train_df.drop("Activity", axis=1)

for feature in zip(feat_labels, model.feature_importances_):
    print(feature)

('tBodyAcc-mean()-X', 0.00020625496923188362)
('tBodyAcc-mean()-Y', 0.00043100226087882076)
('tBodyAcc-mean()-Z', 0.0002866445715581814)
('tBodyAcc-std()-X', 0.007105548790415495)
('tBodyAcc-std()-Y', 0.00042277371699164134)
('tBodyAcc-std()-Z', 0.00011328234187845311)
('tBodyAcc-mad()-X', 0.004793200870594274)
('tBodyAcc-mad()-Y', 0.0010196858379435206)
('tBodyAcc-mad()-Z', 0.00040224461867765257)
('tBodyAcc-max()-X', 0.02269047321505811)
('tBodyAcc-max()-Y', 0.0002721475656957166)
('tBodyAcc-max()-Z', 0.00032748096370956007)
('tBodyAcc-min()-X', 0.0004213561904017142)
('tBodyAcc-min()-Y', 0.0002565091356251912)
('tBodyAcc-min()-Z', 0.000316534590645275)
('tBodyAcc-sma()', 0.0008879435009793585)
('tBodyAcc-energy()-X', 0.005108999629385477)
('tBodyAcc-energy()-Y', 0.0006466001134859161)
('tBodyAcc-energy()-Z', 0.0003574544625917769)
('tBodyAcc-iqr()-X', 0.0015784009857404166)
('tBodyAcc-iqr()-Y', 0.0010197978099412098)
('tBodyAcc-iqr()-Z', 0.00018216439109204253)
('tBodyAcc-entropy()-

In [6]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.0017
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(model, threshold=0.0017)

# Train the selector
sfm.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
        norm_order=1, prefit=False, threshold=0.0017)

In [7]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

print(X_important_train.shape)

(999, 134)


In [8]:
# Create a new random forest classifier for the most important features
clf_important = RandomForestClassifier(n_estimators=500, random_state=0, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, y_train)
clf_important.score(X_important_test, y_test)

0.9049049049049049